In [34]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import numpy as np

# Load the dataset
file_path = 'dataset.xlsx'
data = pd.read_excel(file_path)

# Display the first few rows of the dataset and its summary statistics
data_info = data.info()
data_head = data.head()
data_description = data.describe()

data_info, data_head, data_description


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7818 entries, 0 to 7817
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   No.        7818 non-null   object 
 1   district   7818 non-null   object 
 2   Gini       7693 non-null   float64
 3   country    7818 non-null   object 
 4   state      7818 non-null   object 
 5   year       7818 non-null   int64  
 6   dyid       7818 non-null   object 
 7   potassium  4708 non-null   float64
 8   GDP        7798 non-null   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 549.8+ KB


(None,
   No.       district  Gini country                        state  year  \
 0   1  SOUTH ANDAMAN  0.33   INDIA  ANDAMAN AND NICOBAR ISLANDS  2000   
 1   1  SOUTH ANDAMAN  0.33   INDIA  ANDAMAN AND NICOBAR ISLANDS  2001   
 2   1  SOUTH ANDAMAN  0.33   INDIA  ANDAMAN AND NICOBAR ISLANDS  2002   
 3   1  SOUTH ANDAMAN  0.33   INDIA  ANDAMAN AND NICOBAR ISLANDS  2003   
 4   1  SOUTH ANDAMAN  0.33   INDIA  ANDAMAN AND NICOBAR ISLANDS  2004   
 
                 dyid  potassium          GDP  
 0  SOUTH ANDAMAN2000        NaN  1247.388396  
 1  SOUTH ANDAMAN2001        NaN  1275.670472  
 2  SOUTH ANDAMAN2002        NaN  1411.126730  
 3  SOUTH ANDAMAN2003        NaN  1559.979760  
 4  SOUTH ANDAMAN2004   1.742308  1619.520972  ,
               Gini         year    potassium           GDP
 count  7693.000000  7818.000000  4708.000000  7.798000e+03
 mean      0.266969  2008.966232    15.872150  3.296040e+05
 std       0.055361     5.474388    25.695746  2.769489e+05
 min       0.16000

In [35]:
import statsmodels.api as sm

# Define the independent variables and add a constant to the model (intercept)


def runner(data):
    data_clean = data.dropna(subset=['potassium','GDP','Gini'])
    #imputer = SimpleImputer(strategy='median')
    #data_clean[['GDP', 'Gini']] = imputer.fit_transform(data_clean[['GDP', 'Gini']])

    # Create polynomial features for GDP
    poly = PolynomialFeatures(degree=3, include_bias=False)
    gdp_poly = poly.fit_transform(data_clean[['GDP']])
    gdp_poly_df = pd.DataFrame(gdp_poly, columns=['GDP', 'GDP2', 'GDP3'], index=data_clean.index)

    # Merge polynomial GDP features back into the original dataframe
    data_clean = pd.concat([data_clean, gdp_poly_df[['GDP2', 'GDP3']]], axis=1)

    # Normalize 'GDP', 'Gini', 'potassium', 'GDP2', 'GDP3'
    scaler = StandardScaler()
    data_clean[['GDP', 'Gini', 'potassium', 'GDP2', 'GDP3']] = scaler.fit_transform(data_clean[['GDP', 'Gini', 'potassium', 'GDP2', 'GDP3']])

    # Check for outliers by looking at statistical descriptions
    data_clean_description = data_clean[['GDP', 'Gini', 'potassium', 'GDP2', 'GDP3']].describe()

    data_clean_description

    X = data_clean[['GDP', 'GDP2', 'GDP3', 'Gini']]
    X = sm.add_constant(X)

    # Define the dependent variable
    y = data_clean['potassium']

    # Fit the regression model
    model = sm.OLS(y, X).fit()

    # Get the summary of the regression model
    model_summary = model.summary()
    print(model_summary)


In [36]:
runner(data)


                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     14.52
Date:                Sun, 28 Apr 2024   Prob (F-statistic):           8.73e-12
Time:                        20:43:42   Log-Likelihood:                -6501.1
No. Observations:                4602   AIC:                         1.301e+04
Df Residuals:                    4597   BIC:                         1.304e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        5.99e-18      0.015   4.09e-16      1.0

In [37]:
import pandas as pd
import os

# Load the dataset
data_path = 'dataset.xlsx'
data = pd.read_excel(data_path)

# Pre-extracted regions and states based on the image provided earlier
regions = {'NORTHERN REGION': ['CHANDIGARH', 'DELHI', 'HARYANA', 'HIMACHAL PRADESH', 'JAMMU AND KASHMIR', 'PUNJAB', 'RAJASTHAN'],
           'NORTH-EASTERN REGION': ['ARUNACHAL PRADESH', 'ASSAM', 'MANIPUR', 'MEGHALAYA', 'MIZORAM', 'NAGALAND', 'TRIPURA'],
           'EASTERN REGION': ['ANDAMAN & NICOBAR ISLANDS', 'BIHAR', 'JHARKHAND', 'ODISHA', 'SIKKIM', 'WEST BENGAL'],
           'CENTRAL REGION': ['CHHATTISGARH', 'MADHYA PRADESH', 'UTTAR PRADESH', 'UTTARANCHAL'],
           'WESTERN REGION': ['THE DADRA AND NAGAR HAVELI AND DAMAN AND DIU', 'GOA', 'GUJARAT', 'MAHARASHTRA'],
           'SOUTHERN REGION': ['ANDHRA PRADESH', 'KARNATAKA', 'KERALA', 'LAKSHADWEEP', 'PONDICHERRY', 'TAMILNADU']}

# Create a directory for the output files
output_dir = 'ECO/'
os.makedirs(output_dir, exist_ok=True)

# Function to save the dataframe to an Excel file
def save_to_excel(df, region_name):
    # Replace spaces and slashes for file compatibility
    file_name = region_name.replace(' ', '_').replace('&', 'and') + '.xlsx'
    file_path = output_dir + file_name
    df.to_excel(file_path, index=False)
    return file_path

# Dictionary to store file paths for the newly created excel files
excel_file_paths = {}

# Iterate over the regions dictionary to filter and save the dataframes
for region, states in regions.items():
    # Filter the data for the current region
    df_region = data[data['state'].str.upper().isin(states)]
    # Save to an Excel file
    excel_file_paths[region] = save_to_excel(df_region, region)

excel_file_paths


{'NORTHERN REGION': 'ECO/NORTHERN_REGION.xlsx',
 'NORTH-EASTERN REGION': 'ECO/NORTH-EASTERN_REGION.xlsx',
 'EASTERN REGION': 'ECO/EASTERN_REGION.xlsx',
 'CENTRAL REGION': 'ECO/CENTRAL_REGION.xlsx',
 'WESTERN REGION': 'ECO/WESTERN_REGION.xlsx',
 'SOUTHERN REGION': 'ECO/SOUTHERN_REGION.xlsx'}

In [38]:

data = pd.read_excel('ECO/CENTRAL_REGION.xlsx')

runner(data)



                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.5693
Date:                Sun, 28 Apr 2024   Prob (F-statistic):              0.685
Time:                        20:43:48   Log-Likelihood:                -1738.5
No. Observations:                1226   AIC:                             3487.
Df Residuals:                    1221   BIC:                             3513.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.933e-17      0.029   2.43e-15      1.0

In [39]:

data = pd.read_excel('ECO/NORTHERN_REGION.xlsx')

runner(data)

                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     16.88
Date:                Sun, 28 Apr 2024   Prob (F-statistic):           1.68e-13
Time:                        20:43:49   Log-Likelihood:                -1808.7
No. Observations:                1298   AIC:                             3627.
Df Residuals:                    1293   BIC:                             3653.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.126e-17      0.027  -2.26e-15      1.0

In [40]:

data = pd.read_excel('ECO/EASTERN_REGION.xlsx')

runner(data)

                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     12.12
Date:                Sun, 28 Apr 2024   Prob (F-statistic):           1.56e-09
Time:                        20:43:49   Log-Likelihood:                -976.74
No. Observations:                 705   AIC:                             1963.
Df Residuals:                     700   BIC:                             1986.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.778e-17      0.037   1.31e-15      1.0

In [41]:

data = pd.read_excel('ECO/NORTH-EASTERN_REGION.xlsx')

runner(data)

                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     14.09
Date:                Sun, 28 Apr 2024   Prob (F-statistic):           2.63e-10
Time:                        20:43:50   Log-Likelihood:                -310.52
No. Observations:                 237   AIC:                             631.0
Df Residuals:                     232   BIC:                             648.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.274e-16      0.059   2.16e-15      1.0

In [42]:

data = pd.read_excel('ECO/SOUTHERN_REGION.xlsx')

runner(data)

                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     4.920
Date:                Sun, 28 Apr 2024   Prob (F-statistic):           0.000678
Time:                        20:43:50   Log-Likelihood:                -681.28
No. Observations:                 487   AIC:                             1373.
Df Residuals:                     482   BIC:                             1394.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -7.392e-17      0.045  -1.66e-15      1.0

In [43]:
# Create subsets of the data based on different time periods
data_2000_2004 = data[(data['year'] >= 2000) & (data['year'] <= 2004)]
data_2005_2009 = data[(data['year'] >= 2005) & (data['year'] <= 2009)]
data_2010_2014 = data[(data['year'] >= 2010) & (data['year'] <= 2014)]
data_2015_2018 = data[(data['year'] >= 2015) & (data['year'] <= 2018)]

# Check the size of each subset to confirm correct division
(data_2000_2004.shape, data_2005_2009.shape, data_2010_2014.shape, data_2015_2018.shape)


((225, 9), (225, 9), (223, 9), (180, 9))

In [44]:
runner(data_2000_2004)

                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.419
Date:                Sun, 28 Apr 2024   Prob (F-statistic):              0.235
Time:                        21:15:04   Log-Likelihood:                -124.80
No. Observations:                  90   AIC:                             259.6
Df Residuals:                      85   BIC:                             272.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.714e-17      0.105   9.25e-16      1.0

In [45]:
runner(data_2005_2009)

                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     2.652
Date:                Sun, 28 Apr 2024   Prob (F-statistic):             0.0361
Time:                        21:15:15   Log-Likelihood:                -184.84
No. Observations:                 134   AIC:                             379.7
Df Residuals:                     129   BIC:                             394.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.992e-17      0.085  -3.54e-16      1.0

In [46]:
runner(data_2010_2014)

                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.106
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     3.424
Date:                Sun, 28 Apr 2024   Prob (F-statistic):             0.0110
Time:                        21:15:30   Log-Likelihood:                -164.94
No. Observations:                 121   AIC:                             339.9
Df Residuals:                     116   BIC:                             353.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.083e-17      0.088   1.23e-16      1.0

In [47]:
runner(data_2015_2018)

                            OLS Regression Results                            
Dep. Variable:              potassium   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     2.209
Date:                Sun, 28 Apr 2024   Prob (F-statistic):             0.0712
Time:                        21:15:41   Log-Likelihood:                -197.05
No. Observations:                 142   AIC:                             404.1
Df Residuals:                     137   BIC:                             418.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.158e-17      0.083  -7.44e-16      1.0